In [5]:
# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, transpile
from qiskit.visualization import *

# qiskit-ibmq-provider has been deprecated.
# Please see the Migration Guides in https://ibm.biz/provider_migration_guide for more detail.
from qiskit_ibm_runtime import QiskitRuntimeService, Sampler, Estimator, Session, Options

from qiskit_ibm_runtime.fake_provider import FakeProvider
from qiskit_aer import Aer

from qiskit import QuantumCircuit, transpile
from qiskit_aer import Aer
from qiskit.providers.fake_provider import GenericBackendV2
from qiskit.visualization import plot_histogram
from qiskit.result import Result

import pandas as pd
import numpy as np
import pandas as pd

from qiskit import QuantumCircuit, transpile
from qiskit.providers.fake_provider import *
import random
from qiskit.circuit.library import QFT
from qiskit import QuantumRegister, ClassicalRegister
from qiskit import QuantumCircuit, transpile
from qiskit.providers.fake_provider import *


import lightgbm

In [8]:
from qiskit import QuantumCircuit
from qiskit.circuit import Parameter, Gate


class AD0(Gate):
    def __init__(self, x,y, label=None):
        super().__init__('AD0', 2, [x,y], label=label)
        
        
    
    def _define(self):
        qc = QuantumCircuit(2)
        qc.unitary(self.unitary_mat_0()[0], [0, 1])
       
        self.definition = qc

    
    def kraus_list(self,gamma,t):
        nf= np.exp(-gamma*t)
        k1=[[1,0],[0,np.sqrt(nf)]]
        k2=[[0,np.sqrt(1-nf)],[0,0]]
        return k1,k2
    


    def unitary_mat_0(self):
        gamma = self.params[0]
        t = self.params[1]
        #print(f'gamma_1-{gamma}, time_1-{t}\n')
        k_list=self.kraus_list(gamma,t)
        k0=np.array(k_list[0])
        k1=np.array(k_list[1])
        D0=sqrtm(np.identity(2)-np.matmul(k0.T.conjugate(),k0))
        D0dag=sqrtm(np.identity(2)-np.matmul(k0,k0.T.conjugate()))
        D1=sqrtm(np.identity(2)-np.matmul(k1.T.conjugate(),k1))
        D1dag=sqrtm(np.identity(2)-np.matmul(k1,k1.T.conjugate()))

        z=np.array([[1,0],[0,-1]])
        x=np.array([[0,1],[1,0]])
        y=np.array([[0,-1j],[1j,0]])

        u0= np.kron((x-1j*y)/2,D0)+ np.kron((x+1j*y)/2,D0dag)+np.kron((np.identity(2)+z)/2,k0)-np.kron((np.identity(2)-z)/2,k0.T.conjugate())
        u1= np.kron((x-1j*y)/2,D1)+ np.kron((x+1j*y)/2,D1dag)+np.kron((np.identity(2)+z)/2,k1)-np.kron((np.identity(2)-z)/2,k1.T.conjugate())


        return u0,u1



def quantum_circuit(theta,gamma_1,time_1,gamma_2,time_2,gamma_3,time_3):
    x = ParameterVector('x', length=7)
    qc = QuantumCircuit(3)
    
    qc.ry(theta,0)
    qc.cx(0,1)
    qc.cx(1,2)

    qc.append(AD0(gamma_1,time_1),[0,1])
    qc.append(AD0(gamma_2,time_2),[1,2])
    qc.append(AD0(gamma_3,time_3),[2,0])

    return qc



num_qubits = 3

feature_map=QuantumCircuit(3)
feature_map.ry(np.pi/2,0)
feature_map.cx(0,1)
feature_map.cx(1,2)





feature_map.append(AD0(0.24,4),[0,1])
feature_map.append(AD0(0.24,4),[1,2])
feature_map.append(AD0(0.24,4),[2,0])





feature_map.draw()



┌─────────┐          ┌──────────────┐                ┌──────────────┐
q_0: ┤ Ry(π/2) ├──■───────┤0             ├────────────────┤1             ├
     └─────────┘┌─┴─┐     │  Ad0(0.24,4) │┌──────────────┐│              │
q_1: ───────────┤ X ├──■──┤1             ├┤0             ├┤  Ad0(0.24,4) ├
                └───┘┌─┴─┐└──────────────┘│  Ad0(0.24,4) ││              │
q_2: ────────────────┤ X ├────────────────┤1             ├┤0             ├
                     └───┘                └──────────────┘└──────────────┘

In [11]:
MIN_QUBITS = 3
EXCLUDED_BACKENDS = [
    'fake_burlington', 
    'fake_almaden',                                                              
    'fake_cambridge',                                                
    'fake_boeblingen',                                                          
    'fake_essex',                                                        
    'fake_johannesburg',                                                        
    'fake_london',                                                         
    'fake_melbourne',                                                        
    'fake_poughkeepsie',                                                          
    'fake_rochester',                                                         
    'fake_rueschlikon',                                                        
    'fake_singapore',                                                         
    'fake_tenerife',                                                         
    'fake_tokyo']

In [12]:
def get_filtered_backends():
    """Retrieves available backends and applies filtering criteria."""
    all_backends = FakeProvider().backends()
    return [
        backend 
        for backend in all_backends 
        if backend.configuration().n_qubits >= MIN_QUBITS 
           and backend.name() not in EXCLUDED_BACKENDS
    ]

def calculate_expectation_value(circuit, backend, noise_model):
    """Calculates the expectation value of a circuit on a given backend."""
    transpiled_circuit = transpile(circuit, backend, optimization_level=3)  # Consider a default optimization level, will be overriden in runtime
    job = backend.run(transpiled_circuit, noise_model=noise_model)
    result = job.result()
    return sampled_expectation_value(result.get_counts(circuit), observable1)


In [ ]:
# Define optimization levels to explore
optimization_levels = [0, 1, 2, 3]

# Define #of iterations
num_runs=10

# Define the Pauli observable (consider making this an input parameter)
observable1 = Pauli("ZZZ")



In [13]:
transpiled_circuit = transpile(feature_map, backend='fake_jakarta', optimization_level=2)



AttributeError: 'str' object has no attribute 'configuration'

In [ ]:
backends = get_filtered_backends()
for backend in backends:
    print(f"Processing backend: {backend.name()}")

        # Handle potential missing noise model
    try:
        noise_model = NoiseModel.from_backend(backend)
    except AttributeError:
        print(f"Backend {backend.name} does not support noise model retrieval.")
        noise_model = None

    
    for optimization_level in optimization_levels:
            transpiled_circuit = transpile(feature_map, backend, optimization_level=optimization_level)



        for circuit in circuits_to_transpile:
            for optimization_level in optimization_levels:
                transpiled_circuit = transpile(circuit, backend, optimization_level=optimization_level)
                circuit_name = circuit.name

                # Mapomatic calculations
                deflated_circuit = mm.deflate_circuit(transpiled_circuit)
                layouts = mm.matching_layouts(deflated_circuit, backend)
                mapomatic_scores = mm.evaluate_layouts(deflated_circuit, layouts, backend)
                best_score = max(mapomatic_scores, key=lambda item: item[1])[1]  # Get the best Mapomatic score

                for run_id in range(num_runs):
                    expectation_value = calculate_expectation_value(circuit, backend, noise_model)            
                    writer.writerow([circuit_name, run_id + 1, backend.name(), expectation_value, best_score, optimization_level])
